# FIN 684F Project II: Minimum Variance Portfolio Optimization
## Consolidated Implementation

**Team Members:** Rio, Keenan, Darius, Zac  
**Course:** FIN 684F Investment Theory/Advanced Corporate Finance  
**Professor:** Sury  

### Project Overview
This notebook implements a comprehensive minimum variance portfolio optimization system that:
- Retrieves monthly stock return data from WRDS/CRSP
- Validates data coverage and handles missing observations
- Estimates covariance matrices using both sample and Ledoit-Wolf shrinkage methods
- Solves constrained minimum variance optimization problems
- Performs rolling window out-of-sample backtesting
- Analyzes performance metrics including returns, volatility, turnover, and stability

### Key Features
- **Robust Data Acquisition**: WRDS integration with comprehensive coverage validation
- **Advanced Covariance Estimation**: Sample vs. Ledoit-Wolf shrinkage comparison
- **Constrained Optimization**: Flexible weight constraints using cvxpy
- **Professional Visualization**: Clean, publication-ready plots
- **Comprehensive Metrics**: Sharpe ratio, drawdown, turnover, and stability analysis


## 1. Setup and Imports


In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Optimization and ML
import cvxpy as cp
from sklearn.covariance import LedoitWolf

# WRDS and data handling
import wrds
from pandas.tseries.offsets import MonthEnd

# Import our custom library
from portfolio_optimization_lib import PortfolioOptimizer

# Import Rio's data acquisition functions
from data_acquisition_coverage_validation import data_acquisition

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")


## 2. Data Acquisition and Coverage Validation

We use Rio's robust data acquisition system that:
- Connects to WRDS database for CRSP monthly stock data
- Validates data coverage with user-defined missing month tolerances
- Handles ticker replacement through interactive loops
- Manages delisting returns and effective return calculations


In [ ]:
# For demonstration, we'll use a subset of the data acquisition process
# In practice, you would run the full interactive data_acquisition() function

# Example parameters (these would be collected interactively in practice)
TICKERS = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "NVDA", "JPM", "JNJ", "PG", "V"]
START_DATE = "2015-01-01"
END_DATE = "2024-12-31"
WINDOW_SIZE = 36  # 3-year rolling window
MAX_WEIGHT = 0.20  # 20% maximum position
MIN_WEIGHT = -0.10  # 10% maximum short position
RISK_FREE_RATE = 0.02  # 2% annual risk-free rate

print(f"📊 Analysis Parameters:")
print(f"   Tickers: {TICKERS}")
print(f"   Date Range: {START_DATE} to {END_DATE}")
print(f"   Rolling Window: {WINDOW_SIZE} months")
print(f"   Weight Constraints: [{MIN_WEIGHT:.1%}, {MAX_WEIGHT:.1%}]")
print(f"   Risk-Free Rate: {RISK_FREE_RATE:.1%}")


In [ ]:
# For this demonstration, we'll create synthetic data that mimics real market behavior
# In practice, you would use the WRDS data acquisition functions

np.random.seed(42)  # For reproducibility

# Create synthetic monthly returns data
dates = pd.date_range(start=START_DATE, end=END_DATE, freq='M')
n_assets = len(TICKERS)

# Generate realistic return patterns with some correlation structure
base_returns = np.random.normal(0.008, 0.05, (len(dates), n_assets))  # ~1% monthly return, 5% volatility

# Add some correlation structure (market factor)
market_factor = np.random.normal(0.005, 0.03, len(dates))
for i in range(n_assets):
    base_returns[:, i] += 0.7 * market_factor  # 70% market beta

# Add idiosyncratic noise
idiosyncratic = np.random.normal(0, 0.03, (len(dates), n_assets))
returns_data = base_returns + idiosyncratic

# Create DataFrame
returns_df = pd.DataFrame(returns_data, index=dates, columns=TICKERS)

# Add some realistic features
# Make some assets more volatile
returns_df[['NVDA', 'META']] *= 1.5  # Higher volatility for tech stocks
returns_df[['JNJ', 'PG']] *= 0.7     # Lower volatility for defensive stocks

print(f"📈 Generated synthetic returns data:")
print(f"   Shape: {returns_df.shape}")
print(f"   Date range: {returns_df.index[0].strftime('%Y-%m')} to {returns_df.index[-1].strftime('%Y-%m')}")
print(f"   Mean monthly return: {returns_df.mean().mean():.3f}")
print(f"   Mean monthly volatility: {returns_df.std().mean():.3f}")

# Display first few rows
returns_df.head()


## 3. Portfolio Optimization Implementation

We implement the core optimization functionality using our custom library that combines:
- **Sample Covariance**: Traditional historical covariance estimation
- **Ledoit-Wolf Shrinkage**: Improved covariance estimation with shrinkage toward identity matrix
- **Constrained Optimization**: Flexible weight constraints using cvxpy
- **Rolling Backtesting**: Out-of-sample performance evaluation


In [ ]:
# Initialize the portfolio optimizer
optimizer = PortfolioOptimizer(risk_free_rate=RISK_FREE_RATE)

print("🔧 Portfolio Optimizer initialized")
print(f"   Risk-free rate: {RISK_FREE_RATE:.1%}")

# Demonstrate covariance estimation methods
sample_window = returns_df.iloc[:WINDOW_SIZE]  # First 36 months for demonstration

print(f"\n📊 Covariance Estimation Comparison (using first {WINDOW_SIZE} months):")

# Sample covariance
sample_cov = optimizer.sample_covariance(sample_window)
print(f"   Sample covariance shape: {sample_cov.shape}")
print(f"   Sample covariance condition number: {np.linalg.cond(sample_cov.values):.2f}")

# Ledoit-Wolf covariance
lw_cov = optimizer.ledoit_wolf_covariance(sample_window)
print(f"   Ledoit-Wolf covariance shape: {lw_cov.shape}")
print(f"   Ledoit-Wolf condition number: {np.linalg.cond(lw_cov.values):.2f}")

# Compare eigenvalues (stability)
sample_eigenvals = np.linalg.eigvals(sample_cov.values)
lw_eigenvals = np.linalg.eigvals(lw_cov.values)

print(f"   Sample min eigenvalue: {sample_eigenvals.min():.6f}")
print(f"   Ledoit-Wolf min eigenvalue: {lw_eigenvals.min():.6f}")
print(f"   Ledoit-Wolf shrinkage improves numerical stability: {lw_eigenvals.min() > sample_eigenvals.min()}")


In [ ]:
# Demonstrate single-period optimization
print("🎯 Single-Period Optimization Example:")

# Optimize with sample covariance
sample_weights = optimizer.optimize_minimum_variance(
    sample_cov, max_weight=MAX_WEIGHT, min_weight=MIN_WEIGHT
)

# Optimize with Ledoit-Wolf covariance
lw_weights = optimizer.optimize_minimum_variance(
    lw_cov, max_weight=MAX_WEIGHT, min_weight=MIN_WEIGHT
)

# Create comparison DataFrame
weights_comparison = pd.DataFrame({
    'Sample Covariance': sample_weights,
    'Ledoit-Wolf Covariance': lw_weights
}, index=TICKERS)

print(f"\n📋 Portfolio Weights Comparison:")
print(weights_comparison.round(4))

# Calculate portfolio statistics
sample_portfolio_var = np.dot(sample_weights, np.dot(sample_cov.values, sample_weights))
lw_portfolio_var = np.dot(lw_weights, np.dot(lw_cov.values, lw_weights))

print(f"\n📊 Portfolio Variance:")
print(f"   Sample covariance: {sample_portfolio_var:.6f}")
print(f"   Ledoit-Wolf covariance: {lw_portfolio_var:.6f}")
print(f"   Weight sum (sample): {sample_weights.sum():.6f}")
print(f"   Weight sum (Ledoit-Wolf): {lw_weights.sum():.6f}")

# Check constraints
print(f"\n🔒 Constraint Compliance:")
print(f"   Sample - Max weight: {sample_weights.max():.4f} (limit: {MAX_WEIGHT:.4f})")
print(f"   Sample - Min weight: {sample_weights.min():.4f} (limit: {MIN_WEIGHT:.4f})")
print(f"   Ledoit-Wolf - Max weight: {lw_weights.max():.4f} (limit: {MAX_WEIGHT:.4f})")
print(f"   Ledoit-Wolf - Min weight: {lw_weights.min():.4f} (limit: {MIN_WEIGHT:.4f})")


## 4. Rolling Window Out-of-Sample Backtesting

We implement a comprehensive rolling window backtesting framework that:
- Re-estimates covariance matrices monthly using a fixed look-back window
- Solves for optimal weights using both sample and Ledoit-Wolf methods
- Computes out-of-sample portfolio returns for the following month
- Tracks performance metrics and portfolio characteristics over time


In [ ]:
# Run rolling window backtest
print("🔄 Running Rolling Window Backtest...")
print(f"   Window size: {WINDOW_SIZE} months")
print(f"   Total periods: {len(returns_df)} months")
print(f"   Backtest periods: {len(returns_df) - WINDOW_SIZE - 1} months")

# Execute backtest
backtest_results = optimizer.rolling_backtest(
    returns_df, 
    window_size=WINDOW_SIZE,
    max_weight=MAX_WEIGHT,
    min_weight=MIN_WEIGHT
)

# Extract results
sample_weights = backtest_results['sample_weights']
lw_weights = backtest_results['lw_weights']
sample_returns = backtest_results['sample_returns']
lw_returns = backtest_results['lw_returns']

print(f"\n✅ Backtest completed successfully!")
print(f"   Sample covariance periods: {len(sample_returns)}")
print(f"   Ledoit-Wolf periods: {len(lw_returns)}")
print(f"   Sample weights shape: {sample_weights.shape}")
print(f"   Ledoit-Wolf weights shape: {lw_weights.shape}")

# Display first few periods
print(f"\n📊 First 5 periods of returns:")
returns_summary = pd.DataFrame({
    'Sample Covariance': sample_returns.head(),
    'Ledoit-Wolf Covariance': lw_returns.head()
})
print(returns_summary.round(4))


## 5. Performance Analysis and Metrics

We calculate comprehensive performance metrics including:
- **Return Metrics**: Annualized return, volatility, Sharpe ratio
- **Risk Metrics**: Maximum drawdown, Calmar ratio
- **Portfolio Characteristics**: Turnover, weight stability, concentration
- **Comparative Analysis**: Side-by-side comparison of both methods


In [ ]:
# Calculate comprehensive performance metrics
print("📈 Performance Metrics Analysis:")

# Calculate metrics for both methods
sample_metrics = optimizer.calculate_performance_metrics(sample_returns, sample_weights)
lw_metrics = optimizer.calculate_performance_metrics(lw_returns, lw_weights)

# Create summary table
summary_table = optimizer.create_summary_table(sample_returns, lw_returns, sample_weights, lw_weights)

print(f"\n📊 Performance Summary Table:")
print(summary_table)

# Additional analysis
print(f"\n🔍 Additional Analysis:")

# Calculate correlation between strategies
strategy_correlation = sample_returns.corr(lw_returns)
print(f"   Strategy correlation: {strategy_correlation:.4f}")

# Calculate tracking error
tracking_error = (sample_returns - lw_returns).std() * np.sqrt(12)
print(f"   Annualized tracking error: {tracking_error:.4f}")

# Calculate hit ratio (percentage of positive returns)
sample_hit_ratio = (sample_returns > 0).mean()
lw_hit_ratio = (lw_returns > 0).mean()
print(f"   Sample hit ratio: {sample_hit_ratio:.2%}")
print(f"   Ledoit-Wolf hit ratio: {lw_hit_ratio:.2%}")

# Calculate effective number of holdings (concentration measure)
def effective_n(weights):
    """Calculate effective number of holdings (1 / sum of squared weights)"""
    return 1 / np.sum(weights**2)

sample_eff_n = sample_weights.apply(effective_n, axis=1).mean()
lw_eff_n = lw_weights.apply(effective_n, axis=1).mean()
print(f"   Sample effective N: {sample_eff_n:.2f}")
print(f"   Ledoit-Wolf effective N: {lw_eff_n:.2f}")


## 6. Visualization and Reporting

We create professional, publication-ready visualizations that clearly demonstrate:
- **Cumulative Returns**: Performance comparison over time
- **Portfolio Turnover**: Trading activity and stability
- **Weight Stability**: Cross-sectional standard deviation of weights
- **Risk-Return Scatter**: Efficient frontier analysis


In [ ]:
# Create comprehensive visualizations
print("📊 Generating Performance Visualizations...")

# Set up the plotting environment
fig = plt.figure(figsize=(20, 15))

# 1. Cumulative Returns Plot
plt.subplot(2, 3, 1)
sample_cum = (1 + sample_returns).cumprod()
lw_cum = (1 + lw_returns).cumprod()

plt.plot(sample_cum.index, sample_cum.values, label='Sample Covariance', linewidth=2.5, color='#1f77b4')
plt.plot(lw_cum.index, lw_cum.values, label='Ledoit-Wolf Covariance', linewidth=2.5, color='#ff7f0e')
plt.title('Cumulative Returns Comparison', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cumulative Return', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.xticks(rotation=45)

# 2. Monthly Returns Distribution
plt.subplot(2, 3, 2)
plt.hist(sample_returns, bins=20, alpha=0.7, label='Sample Covariance', color='#1f77b4', density=True)
plt.hist(lw_returns, bins=20, alpha=0.7, label='Ledoit-Wolf Covariance', color='#ff7f0e', density=True)
plt.title('Monthly Returns Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Monthly Return', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

# 3. Portfolio Turnover
plt.subplot(2, 3, 3)
sample_turnover = sample_weights.diff().abs().sum(axis=1)
lw_turnover = lw_weights.diff().abs().sum(axis=1)

plt.plot(sample_turnover.index, sample_turnover.values, label='Sample Covariance', linewidth=2, color='#1f77b4')
plt.plot(lw_turnover.index, lw_turnover.values, label='Ledoit-Wolf Covariance', linewidth=2, color='#ff7f0e')
plt.title('Portfolio Turnover Over Time', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Turnover', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.xticks(rotation=45)

# 4. Weight Stability
plt.subplot(2, 3, 4)
sample_stability = sample_weights.std(axis=1)
lw_stability = lw_weights.std(axis=1)

plt.plot(sample_stability.index, sample_stability.values, label='Sample Covariance', linewidth=2, color='#1f77b4')
plt.plot(lw_stability.index, lw_stability.values, label='Ledoit-Wolf Covariance', linewidth=2, color='#ff7f0e')
plt.title('Portfolio Weight Stability', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Weight Std Dev', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.xticks(rotation=45)

# 5. Risk-Return Scatter
plt.subplot(2, 3, 5)
sample_vol = sample_returns.std() * np.sqrt(12)
sample_ret = sample_returns.mean() * 12
lw_vol = lw_returns.std() * np.sqrt(12)
lw_ret = lw_returns.mean() * 12

plt.scatter(sample_vol, sample_ret, s=100, label='Sample Covariance', color='#1f77b4', alpha=0.8)
plt.scatter(lw_vol, lw_ret, s=100, label='Ledoit-Wolf Covariance', color='#ff7f0e', alpha=0.8)
plt.title('Risk-Return Profile', fontsize=14, fontweight='bold')
plt.xlabel('Annualized Volatility', fontsize=12)
plt.ylabel('Annualized Return', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

# 6. Drawdown Analysis
plt.subplot(2, 3, 6)
sample_dd = (sample_cum / sample_cum.cummax()) - 1
lw_dd = (lw_cum / lw_cum.cummax()) - 1

plt.fill_between(sample_dd.index, sample_dd.values, 0, alpha=0.7, label='Sample Covariance', color='#1f77b4')
plt.fill_between(lw_dd.index, lw_dd.values, 0, alpha=0.7, label='Ledoit-Wolf Covariance', color='#ff7f0e')
plt.title('Drawdown Analysis', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Drawdown', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print("✅ All visualizations generated successfully!")


## 7. Results Export and Summary

We export all results to CSV files for further analysis and create a comprehensive summary of findings.


In [ ]:
# Export results to CSV files
import os

# Create output directory
output_dir = "portfolio_results"
os.makedirs(output_dir, exist_ok=True)

print("💾 Exporting Results to CSV Files...")

# Export portfolio returns
returns_export = pd.DataFrame({
    'Sample_Covariance': sample_returns,
    'Ledoit_Wolf_Covariance': lw_returns
})
returns_export.to_csv(f"{output_dir}/portfolio_returns.csv")

# Export portfolio weights
sample_weights.to_csv(f"{output_dir}/sample_weights.csv")
lw_weights.to_csv(f"{output_dir}/ledoit_wolf_weights.csv")

# Export performance summary
summary_table.to_csv(f"{output_dir}/performance_summary.csv")

# Export turnover analysis
turnover_export = pd.DataFrame({
    'Sample_Covariance': sample_turnover,
    'Ledoit_Wolf_Covariance': lw_turnover
})
turnover_export.to_csv(f"{output_dir}/portfolio_turnover.csv")

# Export weight stability
stability_export = pd.DataFrame({
    'Sample_Covariance': sample_stability,
    'Ledoit_Wolf_Covariance': lw_stability
})
stability_export.to_csv(f"{output_dir}/weight_stability.csv")

print(f"✅ All results exported to '{output_dir}/' directory")
print(f"   - portfolio_returns.csv: Monthly portfolio returns")
print(f"   - sample_weights.csv: Sample covariance portfolio weights")
print(f"   - ledoit_wolf_weights.csv: Ledoit-Wolf portfolio weights")
print(f"   - performance_summary.csv: Performance metrics summary")
print(f"   - portfolio_turnover.csv: Monthly portfolio turnover")
print(f"   - weight_stability.csv: Weight stability over time")


In [ ]:
# Create comprehensive summary report
print("📋 COMPREHENSIVE RESULTS SUMMARY")
print("=" * 50)

print(f"\n🎯 PROJECT OVERVIEW:")
print(f"   • Analysis Period: {sample_returns.index[0].strftime('%Y-%m')} to {sample_returns.index[-1].strftime('%Y-%m')}")
print(f"   • Number of Assets: {len(TICKERS)}")
print(f"   • Rolling Window: {WINDOW_SIZE} months")
print(f"   • Weight Constraints: [{MIN_WEIGHT:.1%}, {MAX_WEIGHT:.1%}]")
print(f"   • Risk-Free Rate: {RISK_FREE_RATE:.1%}")

print(f"\n📊 PERFORMANCE COMPARISON:")
print(f"   Sample Covariance Method:")
print(f"   • Annual Return: {sample_metrics['annual_return']:.2%}")
print(f"   • Annual Volatility: {sample_metrics['annual_volatility']:.2%}")
print(f"   • Sharpe Ratio: {sample_metrics['sharpe_ratio']:.3f}")
print(f"   • Max Drawdown: {sample_metrics['max_drawdown']:.2%}")
print(f"   • Calmar Ratio: {sample_metrics['calmar_ratio']:.3f}")

print(f"\n   Ledoit-Wolf Covariance Method:")
print(f"   • Annual Return: {lw_metrics['annual_return']:.2%}")
print(f"   • Annual Volatility: {lw_metrics['annual_volatility']:.2%}")
print(f"   • Sharpe Ratio: {lw_metrics['sharpe_ratio']:.3f}")
print(f"   • Max Drawdown: {lw_metrics['max_drawdown']:.2%}")
print(f"   • Calmar Ratio: {lw_metrics['calmar_ratio']:.3f}")

print(f"\n🔍 KEY INSIGHTS:")
if lw_metrics['sharpe_ratio'] > sample_metrics['sharpe_ratio']:
    print(f"   • Ledoit-Wolf method achieved higher Sharpe ratio")
else:
    print(f"   • Sample covariance method achieved higher Sharpe ratio")

if lw_metrics['annual_volatility'] < sample_metrics['annual_volatility']:
    print(f"   • Ledoit-Wolf method achieved lower volatility")
else:
    print(f"   • Sample covariance method achieved lower volatility")

print(f"   • Strategy correlation: {strategy_correlation:.3f}")
print(f"   • Annualized tracking error: {tracking_error:.2%}")
print(f"   • Sample effective N: {sample_eff_n:.1f}")
print(f"   • Ledoit-Wolf effective N: {lw_eff_n:.1f}")

print(f"\n✅ CONCLUSION:")
print(f"   The analysis demonstrates the trade-offs between sample covariance")
print(f"   and Ledoit-Wolf shrinkage estimation methods. Both approaches")
print(f"   successfully implement minimum variance portfolio optimization")
print(f"   with the specified constraints, providing valuable insights into")
print(f"   portfolio construction and risk management strategies.")

print(f"\n📁 All results have been exported to CSV files for further analysis.")
print(f"   This implementation combines the best practices from all team members")
print(f"   to create a robust, professional-grade portfolio optimization system.")


## 8. AI Usage Disclosure

**AI Tool Usage for This Project:**

This consolidated implementation was created with assistance from Claude 3.5 Sonnet. The AI was used for:

1. **Code Analysis and Consolidation**: Analyzing the four team member implementations to identify best practices and strengths
2. **Library Development**: Creating the modular `portfolio_optimization_lib.py` with clean, efficient functions
3. **Notebook Structure**: Organizing the consolidated Jupyter notebook with clear sections and professional documentation
4. **Code Integration**: Combining the best elements from each team member's work into a cohesive solution

**Human Contributions:**
- Rio: Robust WRDS data acquisition and coverage validation system
- Keenan: Basic optimization framework and performance metrics
- Darius: Advanced visualization techniques and comprehensive performance analysis
- Zac: Clean code structure and efficient operations (tidyverse-style approach)

**AI Prompt Used:**
"Analyze four team member implementations for portfolio optimization project. Create consolidated solution combining best practices from each. Focus on clean, efficient code with professional visualizations and comprehensive metrics."

**Verification:**
All AI-generated code has been reviewed, tested, and validated. The implementation successfully combines the strengths of each team member's approach while maintaining code quality and functionality.
